# Code Snippet for Verilog Code Generation 
#### -By Aymen

In [2]:
import numpy as np
np.set_printoptions(linewidth=400)
from utils.matmul_optimization import read_index_array
import math

In [3]:
def check_one_hot(matrix):
    return np.all(np.sum(matrix, axis=1) == 1)

In [4]:
def minimize_operation_matrix(matrix, matrix_binarized = False, bit_size = 8):
    """
    Args:
        matrix: given m x n matrix
        bit_size: bit width of each element in the matrix
        matrix_binarized: flag to show whether matrix is already binarized or not
        
    Output:
        index_array: format of array is: [left#, right#, target#, 'operation'] 
            and first row is: [rows, cols, bit, 'parameters']
    """
    # bit_size = bit_size+1
    rows1, cols1 = matrix.shape 
    bin_max_length = cols1*bit_size*3
    bin_weight = cols1*bit_size
    
    # Binarize the matrix
    if not matrix_binarized:
        bin_matrix_ext = np.zeros( (rows1, bin_max_length), dtype=np.uint8)
        # Element wise conversion of each element to binary number for 2^bit notation
        bin_matrix = ((matrix.reshape(-1, 1) & (2**np.arange(bit_size)[::-1])) != 0).astype(int)
        bin_matrix = bin_matrix.reshape(rows1, -1)
        # Copying elements of bin matrix to extended binarized matrix
        bin_matrix_ext[:,:bit_size*cols1] = bin_matrix
    else:
        bin_matrix_ext = matrix
    
    # Start of the algorithm proper , we find the max number of matching one.
    # find the number of max ones of input binary matrix
    max_matching = bin_matrix_ext.sum(axis=0).max()

    
    # process binary matrix
    rows2, cols2 = bin_matrix_ext.shape
    cnt = 0 #count number of matches
    add_count = [0]*rows2 #count number of additions 
    one_count = int(max_matching) # count number of one in column

    # Initialize index_array
    # the first row of index_array is to store required parameters, [rows, cols, bit_size, 'parameters']
    index_array = []
    index_array.append([rows1, cols1, bit_size,'parameters'])
    a = 1
    for i in range(bin_weight):
        if((i%bit_size)==0):
            #print("MEM[%d] = -(input_vector[%d] << %d);" %(i,a-1,a*bit_size-i-1))
            index_array.append([i,a-1,a*bit_size-i-1,'-<<'])
        else:
            #print("MEM[%d] = input_vector[%d] << %d;" %(i,a-1,a*bit_size-i-1))
            index_array.append([i,a-1,a*bit_size-i-1,'<<'])
        if((i+1)%bit_size==0):
            a+=1
    
    # While there is a column with more than one '1' in binary_expanded_matrix:
    while(one_count >= 1):
        #print("\n Max number of 1: %d" % one_count)
        i = 0
        # while(i < cols2 and bin_max_length > bin_weight+cnt):
        while(i < bin_matrix_ext.shape[1]): # and bin_max_length > bin_weight+cnt):
            # doubling the matrix size whenever it runs out of memory  
            if bin_max_length <= bin_weight+cnt+2: 
                print('Increasing matrix size, current bin_max_length:', bin_max_length, 'bin_weight:', bin_weight, 'cnt:', cnt)
                bin_max_length += bin_matrix_ext.shape[1]
                bin_matrix_ext = np.hstack((bin_matrix_ext, np.zeros((rows1, bin_matrix_ext.shape[1]), 
                                                            dtype=np.int32)))
                print('Increasing matrix size, updated bin_max_length:', bin_max_length)
            
            # print('i:', i, 'searching for col:', bin_matrix_ext[:,i])
            # print('one_count:', one_count, 'i:', i)
            if((np.sum(bin_matrix_ext[:,i]==1))>=one_count):
                matching_idx = ((bin_matrix_ext[:,i] @ bin_matrix_ext) == one_count).nonzero()[0]
                # print('matching_idx:', matching_idx)
                if len(matching_idx) > 1:
                    j = matching_idx[0] if i != matching_idx[0] else matching_idx[1]
                    # print('j:', j, 'found col:', bin_matrix_ext[:,j])
                    #print("MEM[%d] = MEM[%d] + MEM[%d];"%(bin_weight+cnt,i,j))
                    index_array.append([i,j,bin_weight+cnt,'+'])

                    bin_matrix_ext[:,bin_weight+cnt] = bin_matrix_ext[:,i]&bin_matrix_ext[:,j]
                    bin_matrix_ext[:,i] = bin_matrix_ext[:,i]&(~bin_matrix_ext[:,bin_weight+cnt])
                    bin_matrix_ext[:,j] = bin_matrix_ext[:,j]&(~bin_matrix_ext[:,bin_weight+cnt])


                    cnt += 1
                    add_count[rows2-one_count] += 1
                else:
                    i += 1
            else:
                i += 1
        
        if check_one_hot(bin_matrix_ext):
            break
        #print("Counts of Additions for %d one match:  %d" %(one_count,add_count[rows2-one_count]))
        one_count=one_count-1
    
    # Adding non zero operations at the end
    nonzero_row_cols = bin_matrix_ext.nonzero() 
    nonzero_rows = nonzero_row_cols[0]
    nonzero_cols = nonzero_row_cols[1]
    index_array = index_array + [[row, col, -1, '='] for row, col in zip(nonzero_rows, nonzero_cols)]
    
    return index_array, bin_matrix_ext

In [5]:
# index_ array contient deja N_rows et N_columns, et n'a pas beoins de n_bits_weights, mais on a besoin de n_bits_input pour la structure du module 
def generate_verilog_optimized(index_array, input_bitwidth = 6, filename = ''):
    # define parameters
    rows1, cols1 = index_array[0][0], index_array[0][1]
    weight_bitwidth = index_array[0][2]
    output_bitwidth = weight_bitwidth + input_bitwidth + math.ceil(math.log(cols1 - 1 ,2))
    mem_size = max(row[2] for row in index_array)
    
    added_statements = ""
    for row in index_array[1:]:
        if row[2] != -1:
            if row[3] == '<<':
                added_statements += f"assign MEM[{row[0]}] = input_vector[{row[1]}] {row[3]} {row[2]};\n"
            elif row[3] == '-<<':
                added_statements += f"assign MEM[{row[0]}] = -(input_vector[{row[1]}] << {row[2]});\n"
            # actually, we don't consider >> operation here.
            elif row[3] == '>>':
                added_statements += f"assign MEM[{row[0]}] = input_vector[{row[1]}] {row[3]} {row[2]};\n"
            elif row[3] == '+':
                added_statements += f"assign MEM[{row[2]}] = MEM[{row[0]}] {row[3]} MEM[{row[1]}];\n"
        else:
            added_statements += f"assign output_vector[{row[0]}] = MEM[{row[1]}];\n"    
    
    statements = [
        "// Verilog module\n",
        f"module matrix_mult_optimized_{rows1}x{cols1}_{weight_bitwidth}_{mem_size}#(\n",
        f"    parameter ROWS = {rows1},\n",
        f"    parameter COLS = {cols1},\n",
        f"    parameter MEM_SIZE = {mem_size},\n",
        f"    parameter input_bit_width = {input_bitwidth},\n",
        f"    parameter output_bit_width = {output_bitwidth}\n",
        ")(\n",
        "    input wire signed [input_bit_width-1:0] input_vector [0: COLS-1],\n",
        f"   output wire signed [output_bit_width-1:0] output_vector [0: ROWS-1]\n",
        ");\n\n",
        f"wire signed [output_bit_width-1:0] MEM [0:MEM_SIZE];\n\n",
        added_statements,
        "\n",
        "endmodule\n"
    ]
    # Create and open a new Verilog file
    # verilog_file = open(f"output/matrix_multiplier_{mem_size}.sv", "w")
    verilog_file = open(f"Verilog_Files/matrix_mult_optimized_{filename}{rows1}x{cols1}_{weight_bitwidth}_{mem_size}.sv", "w")

    # Write the strings to the Verilog file
    for statement in statements:
        verilog_file.write(statement)
    # Close the Verilog file
    verilog_file.close()
    print("Successfully generate verilog file!")
    return verilog_file, weight_bitwidth

##### Defining Model

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data import DataLoader
import brevitas.nn as qnn

# Define the transformation for dataset (if needed)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Load the training and test datasets
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Create data loaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.quant_inp1 = qnn.QuantIdentity(bit_width=8, signed=False, return_quant_tensor=True)
        self.fc1 = qnn.QuantLinear(784, 64, bias=False,
                                   weight_bit_width=8,
                                   bias_bit_width=8)
        self.relu1 = qnn.QuantReLU(bit_width=26, return_quant_tensor=True)
        self.quant_inp2 = qnn.QuantIdentity(bit_width=8, signed=False, return_quant_tensor=True)
        self.fc2 = qnn.QuantLinear(64, 64, bias=False,
                                   weight_bit_width=8,
                                   bias_bit_width=8)
        self.relu2 = qnn.QuantReLU(bit_width=22, return_quant_tensor=True)
        self.quant_inp3 = qnn.QuantIdentity(bit_width=8, signed=False, return_quant_tensor=True)
        self.fc3 = qnn.QuantLinear(64, 64, bias=False,
                                   weight_bit_width=8,
                                   bias_bit_width=8)
        self.relu3 = qnn.QuantReLU(bit_width=22, return_quant_tensor=True)
        self.quant_inp4 = qnn.QuantIdentity(bit_width=8, signed=False, return_quant_tensor=True)
        self.fc4 = qnn.QuantLinear(64, 10, bias=False,
                                   weight_bit_width=8,
                                   bias_bit_width=8)
        self.relu4 = qnn.QuantReLU(bit_width=22, return_quant_tensor=True)
        self.quant_inp5 = qnn.QuantIdentity(bit_width=8, signed=False, return_quant_tensor=True)
        
    def forward(self, x):
        x = torch.flatten(x, 1)  # Flatten the input tensor
        x = self.quant_inp1(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.quant_inp2(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.quant_inp3(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.quant_inp4(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.quant_inp5(x)
        return x
    
    #new function to extract activations after each layer

    def forward_compute(self, x):
        x = torch.flatten(x, 1)  # Flatten the input tensor
        y = []
        z = []
        x = self.quant_inp1(x)
        y.append(x)
        x = self.fc1(x)
        x = self.relu1(x)
        y.append(x)
        x = self.quant_inp2(x)
        z.append(x)
        x = self.fc2(x)
        x = self.relu2(x)
        y.append(x)
        x = self.quant_inp3(x)
        z.append(x)
        x = self.fc3(x)
        x = self.relu3(x)
        y.append(x)
        x = self.quant_inp4(x)
        z.append(x)
        x = self.fc4(x)
        x = self.relu4(x)
        y.append(x)
        x = self.quant_inp5(x)
        z.append(x)
        return x,y,z


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [7]:
test_data = datasets.MNIST(root='./data', train=False, download=True)
data, target = test_data[0]
data

In [8]:
# # Load the FP32 model weights
# model_fp32_weights = torch.load('model_fp32.pth')

# Initialize the quantized model
model = SimpleNN()

# # Copy the weights from FP32 model to quantized model
# model.fc1.weight.data = model_fp32_weights['fc1.weight'].clone().detach()
# model.fc2.weight.data = model_fp32_weights['fc2.weight'].clone().detach()
# model.fc3.weight.data = model_fp32_weights['fc3.weight'].clone().detach()
# model.fc4.weight.data = model_fp32_weights['fc4.weight'].clone().detach()

In [9]:
model.load_state_dict(torch.load('model_weights_quantized.pth', map_location='cpu'))

<All keys matched successfully>

Defining some of the Helper Functions which we will need in further code.

In [10]:
def to_binary(val, bitwidth):
    if val < 0:
        val = (1 << bitwidth) + val
    return f"{val:0{bitwidth}b}"

In [11]:
def float_to_int_n(values, scale, bit_width, signed=True):
    # Calculate the scaled values
    scaled_values = values / scale
    scaled_values = np.round(scaled_values)
    
    if(signed == True):
        # Determine the range for n-bit signed integers
        int_n_min = -2**(bit_width - 1)
        int_n_max = 2**(bit_width - 1) - 1
        
        # Clip the values to fit within n-bit signed integer range
        scaled_values = np.clip(scaled_values, int_n_min, int_n_max)
        
        # Convert to n-bit integers by truncating to fit in n-bits
        int_n_values = scaled_values.astype(np.int32)  # Initially convert to 32-bit integers
        
        # Manually handle the n-bit integer range
        int_n_values = int_n_values & ((1 << bit_width) - 1)  # Mask to n bits
        
        # Handle sign extension for negative values
        sign_bit = 1 << (bit_width - 1)
        int_n_values = np.where(int_n_values & sign_bit, int_n_values | ~((1 << bit_width) - 1), int_n_values)
        
    else:
        # Determine the range for n-bit unsigned integers
        int_n_min = 0
        int_n_max = 2**bit_width - 1

        # Clip the values to fit within n-bit unsigned integer range
        scaled_values = np.clip(scaled_values, int_n_min, int_n_max)

        # Convert to n-bit integers by truncating to fit in n-bits
        int_n_values = scaled_values.astype(np.uint32)  # Initially convert to 32-bit unsigned integers

        # Manually handle the n-bit integer range
        int_n_values = int_n_values & ((1 << bit_width) - 1)  # Mask to n bits
        
    
    return int_n_values

# Generating the Verilog Codes for different Layers

In [12]:
# Evaluate the model
model.eval()  # Set the model to evaluation mode
data, targets = train_dataset[0]
data.shape

torch.Size([1, 28, 28])

In [13]:
# re-evaluating the model for generating outputs
with torch.no_grad():
    out, outputs, quants = model.forward_compute(data)
    _, predicted = torch.max(out.value, 1)

/workspace/myenv/lib/python3.10/site-packages/torch/_tensor.py:1419: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at ../c10/core/TensorImpl.h:1921.)
  return super().rename(names)


Extracting integer Weights from Model 

In [21]:
# uncomment the code to generate the verilog codes


bitwidths = 8
layers = [0,1,2,3] #add the layers which you want to generate for

count = 0

# Assuming 'model' is already defined and loaded
for layer in layers:
    for name, layer in model.named_modules():
        # Check if the layer is a Linear layer
        if isinstance(layer, nn.Linear):
            
            # Extract the weight matrix, move to CPU, detach from the computation graph, and convert to NumPy array
            if(count == 3):
                int_weights = layer.int_weight().detach().cpu().numpy()
                
                index_array, _ = minimize_operation_matrix(int_weights, bit_size = bitwidths)
                verilog_code, _ = generate_verilog_optimized(index_array , bitwidths)
                
            count = count + 1
            

     

Successfully generate verilog file!


TestBench Generation for Verification of Outputs

In [30]:
# Assuming the output is the same as mentioned
output_values = outputs[0].value.numpy().flatten()
scale = outputs[0].scale.item()
bitwidth = 9

# Convert the floating point values to 8-bit signed integers
input_vector = float_to_int_n(output_values, scale, bitwidth, False)

# Print the input vector for Verilog
input_vector_verilog = [to_binary(val, bitwidth) for val in input_vector]

input_vector_verilog

['000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '000000000',
 '0000

In [31]:
# Generate the Verilog testbench code
verilog_code = f"""
`timescale 1ns / 1ps

module Testbench#(
    parameter ROWS = 64,
    parameter COLS = 64,
    parameter input_bit_width = {bitwidth},
    parameter output_bit_width = {int(outputs[1].bit_width.item())}
)();
        
    logic signed [input_bit_width-1:0] input_vector [0: COLS-1];
    logic signed [output_bit_width-1:0] output_vector [0: ROWS-1];
        
    DNN_Layer test1(
        .input_vector(input_vector),
        .output_vector(output_vector)
    );
        
    initial begin
        // Initialize input_vector with the values from Python
"""

# Add the input vector initialization in Verilog format
for i, val in enumerate(input_vector_verilog):
    verilog_code += f"        input_vector[{i}] = 9'b{val};\n"

# Finish the Verilog code
verilog_code += """
        // Wait for some time to observe the output
        #10;

        // Test the output against the expected values
        // Example: assert(output_vector[0] == expected_value);
        // Add your checks here

        // Finish the simulation
        $finish;
    end
endmodule
"""

# Write the Verilog code to a file
with open("Verilog_Files/testbench.v", "w") as file:
    file.write(verilog_code)

print("Verilog testbench generated and saved as 'testbench.v'")


Verilog testbench generated and saved as 'testbench.v'


# Output Verification

In [32]:
# Extract and print integer weights and scales for specified layers
layers = [0]
count = 0
for name, module in model.named_modules():
    if isinstance(module, qnn.QuantLinear):
        if count in layers:
            int_weights = module.int_weight().detach().cpu().numpy()
            weight_scale = module.quant_weight_scale().detach().cpu().numpy() if hasattr(module, 'quant_weight_scale') else None
            print(module.weight)
            print(f'Layer {count} - Integer Weights:')
            print(int_weights)
            print(f'Weights Shape: {int_weights.shape}')
            print(f'Weight Scale: {weight_scale}')
        
        count += 1

Parameter containing:
tensor([[ 0.0339, -0.0129, -0.0118,  ...,  0.0244, -0.0331,  0.0150],
        [ 0.0200,  0.0345,  0.0050,  ..., -0.0131, -0.0158, -0.0032],
        [-0.0068,  0.0080,  0.0316,  ...,  0.0113, -0.0013, -0.0168],
        ...,
        [-0.0328, -0.0090, -0.0313,  ..., -0.0213, -0.0153,  0.0156],
        [-0.0254,  0.0106, -0.0115,  ..., -0.0043, -0.0261, -0.0339],
        [-0.0116,  0.0351, -0.0144,  ...,  0.0027, -0.0039,  0.0100]],
       requires_grad=True)
Layer 0 - Integer Weights:
[[ 6 -2 -2 ...  4 -6  3]
 [ 3  6  1 ... -2 -3 -1]
 [-1  1  5 ...  2  0 -3]
 ...
 [-6 -2 -5 ... -4 -3  3]
 [-4  2 -2 ... -1 -4 -6]
 [-2  6 -2 ...  0 -1  2]]
Weights Shape: (64, 784)
Weight Scale: 0.005887963343411684


In [33]:
outputs[1]/ (outputs[1].scale.item())

tensor([[26804492., 13827634.,  8363090.,        0.,  5796438.,  4615285.,
                0.,  8603659., 24962580., 12776851., 21841688., 15636567.,
          2828695.,        0.,        0.,        0.,        0.,  9783727.,
         16872168.,        0.,        0.,        0.,        0.,  5260201.,
                0., 13795961.,        0.,        0.,        0., 12243000.,
         26171288.,  6530076.,        0.,        0.,        0.,        0.,
                0., 13628497.,        0.,        0.,        0.,  2786830.,
         33396826.,   619320., 14350421., 11705895., 10250332.,        0.,
                0.,  9115816., 10247077., 41936924.,        0.,  6464784.,
                0.,        0.,        0.,  6642662., 23828068., 19791752.,
          4429164.,        0.,  5322459.,        0.]])

In [34]:
# Assuming the output is the same as mentioned
output_values = outputs[1].value.numpy().flatten()
scale = outputs[0].scale.item()*weight_scale
bitwidth = int(outputs[0].bit_width.item())

# Convert the floating point values to 8-bit signed integers
vec = float_to_int_n(output_values, scale, 26, False)

vec

array([123566,  63744,  38553,      0,  26721,  21276,      0,  39662, 115075,  58900, 100688,  72083,  13040,      0,      0,      0,      0,  45102,  77779,      0,      0,      0,      0,  24249,      0,  63598,      0,      0,      0,  56439, 120647,  30103,      0,      0,      0,      0,      0,  62826,      0,      0,      0,  12847, 153956,   2855,  66154,  53963,  47253,      0,      0,
        42023,  47238, 193325,      0,  29802,      0,      0,      0,  30622, 109845,  91238,  20418,      0,  24536,      0], dtype=uint32)

# Verilog Code Generation for Intermediate Scale Multiplication

In [35]:
# Extract the weight scale of all the layers
count = 0
for name, module in model.named_modules():
    if isinstance(module, qnn.QuantLinear):
        if count == 0:
            weight_scale = module.quant_weight_scale().detach().cpu().numpy() if hasattr(module, 'quant_weight_scale') else None
        count +=1  

In [36]:
#extracting the values
number = (quants[0].scale.item() * weight_scale)/ (outputs[1].scale.item())
number

1472.022871883505

In [37]:
def round_off(number, decimals=0):
    """
    Rounds off a number to a specified number of decimal places.

    Args:
    number (float): The number to be rounded off.
    decimals (int): The number of decimal places to round off to.

    Returns:
    float: The rounded off number.
    """
    multiplier = 10 ** decimals
    return round(number * multiplier) / multiplier

In [38]:
def int_to_binary(num):
    if num < 0:
        raise ValueError("Only non-negative integers are supported.")
    return bin(num).replace("0b", "")

In [39]:
number = int(round(number))

In [40]:
binary_representation = int_to_binary(number)
print(f"The binary representation of {number} is {binary_representation}")

The binary representation of 1472 is 10111000000


In [41]:
verilog_code = '''

mem[0] = in[i] << 5;
mem[1] = in[i] << 4;
mem[2] = in[i] << 3;
mem[3] = in[i] << 2;
mem[4] = in[i] << 1;
mem[5] = mem[0] + mem[1];
mem[6] = mem[2] + mem[3];
mem[7] = mem[4] + mem[5];
mem[8] = mem[6] + mem[7];


''' 

# Testing

In [19]:
# Extract and print integer weights and scales for specified layers
layers = [0]
count = 0
for name, module in model.named_modules():
    if isinstance(module, qnn.QuantLinear):
        if count in layers:
            int_weights = module.int_weight().detach().cpu().numpy()
            absolute_values = np.abs(int_weights)
            row_sums = np.sum(absolute_values, axis=1)
            number = row_sums*127
            max_value = np.max(number)
            num = np.log2(max_value)
            print(max_value)
        count += 1

1673479


In [20]:
# Evaluate the model
model.eval() 

max_values_layer1 = []
max_values_layer2 = []
max_values_layer3 = []
max_value = 0

# re-evaluating the model for generating outputs
with torch.no_grad():
    for data, targets in test_dataset:
        out, outputs, quants = model.forward_compute(data)
        count = 0
        for name, module in model.named_modules():
            if isinstance(module, qnn.QuantLinear):
                if count == 0:
                    weight_scale = module.quant_weight_scale().detach().cpu().numpy() if hasattr(module, 'quant_weight_scale') else None
                    result_tensor = outputs[1] / (outputs[0].scale.item() * weight_scale)
                    result_array = result_tensor.numpy()
                    max_value = np.max(result_array)
                    max_values_layer1.append(max_value)
                if count == 1:
                    weight_scale = module.quant_weight_scale().detach().cpu().numpy() if hasattr(module, 'quant_weight_scale') else None
                    result_tensor = outputs[2] / (quants[0].scale.item() * weight_scale)
                    result_array = result_tensor.numpy()
                    max_value = np.max(result_array)
                    max_values_layer2.append(max_value)
                if count == 2:
                    weight_scale = module.quant_weight_scale().detach().cpu().numpy() if hasattr(module, 'quant_weight_scale') else None
                    result_tensor = outputs[3] / (quants[1].scale.item() * weight_scale)
                    result_array = result_tensor.numpy()
                    max_value = np.max(result_array)
                    max_values_layer3.append(max_value)
                
                count += 1
    

/workspace/myenv/lib/python3.10/site-packages/torch/_tensor.py:1419: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at ../c10/core/TensorImpl.h:1921.)
  return super().rename(names)


KeyboardInterrupt: 

In [76]:
max = 0.01
for i in max_values_layer1:
    if max < i:
        max = i
print("layer1 :", max, "bits:", np.log2(max))

layer1 : 315220.75 bits: 18.266003


In [77]:
max = 0.01
for i in max_values_layer2:
    if max < i:
        max = i
print("layer2 :", max, "bits:", np.log2(max))

layer2 : 67630.68 bits: 16.045391


In [78]:
max = 0.01
for i in max_values_layer3:
    if max < i:
        max = i
print("layer2 :", max, "bits:", np.log2(max))

layer2 : 72450.16 bits: 16.144701


In [45]:
mismatch_indices = np.where(input_vector != rounded_vector)[0]

print("Indices where values do not match:", mismatch_indices)

Indices where values do not match: []


# Comparing the outputs of FPGA and pyTorch

In [42]:
# Evaluate the model
model.eval()  # Set the model to evaluation mode
data, targets = train_dataset[0]
data.shape

print(targets)

# re-evaluating the model for generating outputs
with torch.no_grad():
    out, outputs, quants = model.forward_compute(data)
    _, predicted = torch.max(out.value, 1)

5


In [43]:
## pytorch outputs


layers = [1, 2, 3, 4]
bitwidth = 8

for i in layers:
    if(i==1):
        #pytorch outputs
        output_values = quants[0].value.numpy().flatten()
        scale = quants[0].scale.item()
        layer1_pytorch = float_to_int_n(output_values, scale, bitwidth, False)
        
    if(i==2):
        #pytorch outputs
        output_values = quants[1].value.numpy().flatten()
        scale = quants[1].scale.item()
        layer2_pytorch = float_to_int_n(output_values, scale, bitwidth, False)
        
    if(i==3):
        #pytorch outputs
        output_values = quants[2].value.numpy().flatten()
        scale = quants[2].scale.item()
        layer3_pytorch = float_to_int_n(output_values, scale, bitwidth, False)
    
    if(i==4):
        #pytorch outputs
        output_values = quants[3].value.numpy().flatten()
        scale = quants[3].scale.item()
        layer4_pytorch = float_to_int_n(output_values, scale, bitwidth, False)
        
        

In [52]:
#FPGA simulation results

layers = [0, 1, 2, 3]
count = 0
bitwidth = 8
for name, module in model.named_modules():
    if isinstance(module, qnn.QuantLinear):
        if count in layers:
            int_weights = module.int_weight().detach().cpu().numpy()
            weight_scale = module.quant_weight_scale().detach().cpu().numpy() if hasattr(module, 'quant_weight_scale') else None
            
            if count == 0:
                output_values = outputs[1].value.numpy().flatten()
                scale = outputs[0].scale.item()*weight_scale
                bitwidth = int(outputs[1].bit_width.item())
                input_vector = float_to_int_n(output_values, scale, bitwidth, False) /(2**10)
                out = input_vector.astype(int)
                out[out<0] = 0
                FPGA_outputlayer1 = out
            
            if count == 1:
                input_vector = np.dot(int_weights, FPGA_outputlayer1) / (2**8)
                out = input_vector.astype(int)
                out[out<0] = 0
                FPGA_outputlayer2 = out
                
            if count == 2:
                input_vector = np.dot(int_weights, FPGA_outputlayer2) / (2**8)
                out = input_vector.astype(int)
                out[out<0] = 0
                FPGA_outputlayer3 = out
                
            if count == 3:
                input_vector = np.dot(int_weights, FPGA_outputlayer3) / (2**8)
                out = input_vector.astype(int)
                out[out<0] = 0
                FPGA_outputlayer4 = out
            
        count += 1

In [53]:
import pandas as pd

# Create a DataFrame with the two arrays
df = pd.DataFrame({'pyTorch': layer4_pytorch, 'Vivado': FPGA_outputlayer4})

# Print the DataFrame
print(df)

# Store the DataFrame in a text file
df.to_csv('output.txt', sep='\t', index=False)

   pyTorch  Vivado
0        0       0
1        0       0
2        0       0
3      115     101
4        0       0
5      255     238
6        0       0
7        0       0
8        0       0
9        0       0


In [54]:
def get_pytorch_quantized_outputs(quants, layers, bitwidth=8):
    outputs = {}
    
    for i in layers:
        if i in range(1, 5):
            # PyTorch outputs
            output_values = quants[i-1].value.numpy().flatten()
            scale = quants[i-1].scale.item()
            outputs[f'layer{i}_pytorch'] = float_to_int_n(output_values, scale, bitwidth, False)
    
    return outputs


In [55]:
import numpy as np

def get_fpga_simulation_outputs(model, outputs, bitwidth=8):
    count = 0
    results = {}
    
    for name, module in model.named_modules():
        if isinstance(module, qnn.QuantLinear):
                int_weights = module.int_weight().detach().cpu().numpy()
                weight_scale = module.quant_weight_scale().detach().cpu().numpy() if hasattr(module, 'quant_weight_scale') else None

                if count == 0:
                    output_values = outputs[1].value.numpy().flatten()
                    scale = outputs[0].scale.item() * weight_scale
                    bitwidth = int(outputs[1].bit_width.item())
                    input_vector = float_to_int_n(output_values, scale, bitwidth, False) / (2**10)
                    out = input_vector.astype(int)
                    out[out < 0] = 0
                    results['FPGA_outputlayer1'] = out

                if count == 1:
                    input_vector = np.dot(int_weights, results['FPGA_outputlayer1']) / (2**8)
                    out = input_vector.astype(int)
                    out[out < 0] = 0
                    results['FPGA_outputlayer2'] = out

                if count == 2:
                    input_vector = np.dot(int_weights, results['FPGA_outputlayer2']) / (2**8)
                    out = input_vector.astype(int)
                    out[out < 0] = 0
                    results['FPGA_outputlayer3'] = out

                if count == 3:
                    input_vector = np.dot(int_weights, results['FPGA_outputlayer3']) / (2**8)
                    out = input_vector.astype(int)
                    out[out < 0] = 0
                    results['FPGA_outputlayer4'] = out

                count += 1

    return results



In [17]:
layers = [4]

pyOut = get_pytorch_quantized_outputs(quants, layers, bitwidth=8)
pyOut

{'layer4_pytorch': array([255,   0,  28,   0,   0,   0,  17,   0,   0,   5], dtype=uint32)}

In [18]:
vivadoOut = get_fpga_simulation_outputs(model, outputs)
vivadoOut

{'FPGA_outputlayer1': array([ 68,   0,   0,   0,   0,   0,   0,  70,  64,   0,   2,   0,   0,  24,  35,   0,   0,  32,   9,   0,   0,   0,   0,   0, 163,  81,   0, 120,   0, 163,   0,   4,  93,  70,  68,   0,   0,  25,   0,   0,   0,   0, 106,   0,  11,   0,   0,  44, 118,   0,   0,   0,   0,   0,   0,   0, 142,   0,   0, 137,   0,   0,   0,  53]),
 'FPGA_outputlayer2': array([  0,   0,  33,  99,   0,   0, 118,  45,   0,  48,  14,   0,   0,   0,   0,   0,   0,  17,  17,   0,   0,  75,   9,  57,   0,  36,   0,   0,   0,   0,  68,  83,  65,  15,   0,  54,   0, 125,  39,   0,   0,   0,   0,   0,   7,   0,   0,   0,  40,   0,  10, 156,  21,  31,   0,   0,  36,  75,   9,  98,  11,  76,  28,  61]),
 'FPGA_outputlayer3': array([  0,   3,   2,   0,   0,  76,   0,   0,  74,  60,   0, 111,   0,   0,   0,   1,   0,  26,  40, 108,  56,  35,   0,  89,   0,  74,  19,  10,  81,   0,  82,   0,   0,   0,  39,   0,  58,   7,   0,   7,  39,   0,   0,   0,  90,  13,  74,  64, 112,   0,  82,   0,   0,  46,

# Testing Entire Training + Testing Dataset Images

In [56]:
def evaluate_model_with_detailed_analysis(model, dataset, bitwidth=8):
    model.eval()  # Set the model to evaluation mode

    total_match = 0
    total_samples = 0
    layers = [4]
    index = []
    with torch.no_grad():
        for data, targets in dataset:
            out, outputs, quants = model.forward_compute(data)
            _, predicted = torch.max(out.value, 1)
            pyOut = get_pytorch_quantized_outputs(quants, layers, bitwidth)
            vivadoOut = get_fpga_simulation_outputs(model, outputs)
            py =  pyOut['layer4_pytorch']
            viv = vivadoOut['FPGA_outputlayer4']          
            
            pyClass =  np.argmax(py)
            vivClass = np.argmax(viv)
            
            if(pyClass == vivClass):
                total_match = total_match + 1
            else:
                index.append(total_samples)
                
        
            total_samples += 1

    return index,total_match, total_samples

In [25]:
index, total, match = evaluate_model_with_detailed_analysis(model, train_dataset, bitwidth=8)

In [59]:
import torch
import numpy as np

def evaluate_model_with_detailed_analysis(model, dataset, indices, output_file, bitwidth=8):
    model.eval()  # Set the model to evaluation mode

    total_match = 0
    total_samples = 0
    layers = [4]
    index = []

    with torch.no_grad():
        with open(output_file, 'w') as f:
            for i in indices:
                data, targets = dataset[i]
                out, outputs, quants = model.forward_compute(data)
                _, predicted = torch.max(out.value, 1)
                pyOut = get_pytorch_quantized_outputs(quants, layers, bitwidth)
                vivadoOut = get_fpga_simulation_outputs(model, outputs)
                py = pyOut['layer4_pytorch']
                viv = vivadoOut['FPGA_outputlayer4']

                pyClass = np.argmax(py)
                vivClass = np.argmax(viv)

                f.write(f"Index: {i}\n")
                f.write(f"python: {py} vivado: {viv}\n\n")
                # f.write(f"vivado: {viv}\n\n")

                if pyClass == vivClass:
                    total_match += 1
                else:
                    index.append(total_samples)

                total_samples += 1

    return index, total_match, total_samples

# Assuming train_dataset is the original dataset and indices is the array containing the given indices
indices = [1674, 2734, 3065, 6418, 10852, 13234, 15741, 16446, 16658, 17728, 18966, 22270, 28632, 29609, 31347, 32573, 34785, 35382, 41897, 42986, 45925, 45930, 48166, 50091, 50239, 51544, 53578, 55078, 56842, 57302, 57982, 58822]

output_file = "output.txt"

index, total_match, total_samples = evaluate_model_with_detailed_analysis(model, train_dataset, indices, output_file)

print(f"Indices where classes did not match: {index}")
print(f"Total matches: {total_match}")
print(f"Total samples: {total_samples}")


Indices where classes did not match: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Total matches: 0
Total samples: 32
